In [ ]:
# config
import os
PROJECT_ID = 'PROJECT_ID'  # change here
BUCKET = PROJECT_ID + '_accelerator_demo'
REGION = 'us-central1'
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["BUCKET"] = BUCKET

!gcloud config set project {PROJECT_ID}
!gsutil mb gs://{BUCKET}
!gcloud config set compute/region {REGION}

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
%%bash
mkdir train
echo "" > train/__init__.py
mkdir config

In [ ]:
%%writefile train/model_definition.py
import tensorflow as tf

def create_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

# Train on single Processor

In [ ]:
%%writefile train/train_single_cpu_gpu.py
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

#Train model

model = model_definition.create_model(input_shape=x_train.shape[1:])
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
  loss='sparse_categorical_crossentropy',
  metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with single GPUs: {}".format(time.time() - start))

model.save_weights('./fashion_mnist_single.h5', overwrite=True)

## Submit single worker with CPU

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="single_cpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_single_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \

## Submit single worker with single GPU

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="single_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_single_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --scale-tier=BASIC_GPU

# Train on multiple GPUs

In [ ]:
%%writefile train/train_mult_cpu_gpu.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Multiple GPUs or CPUs ###################
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
###############################################################
    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with multiple GPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_mult_gpu.h5', overwrite=True)

In [ ]:
%%writefile config/multi_gpu_config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 4 T4 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/multi_gpu_config.yaml

# Example training on TPU

In [ ]:
%%writefile train/train_tpu.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Run on TPUs ###################
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
###############################################################
    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with TPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_tpu.h5', overwrite=True)

## Submit single worker with TPUv2 Pod (8 cores)

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="tpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_tpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --scale-tier=BASIC_TPU \
  --region=us-central1

## Submit single worker TPUv3 (8 cores)

In [ ]:
%%writefile config/tpuv3_config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highcpu-16
  workerType: cloud_tpu
  workerCount: 1
  workerConfig:
    acceleratorConfig:
      type: TPU_V3
      count: 8

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="tpu_v3_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_tpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/tpuv3_config.yaml

# Train on multiple device with GPUs

In [ ]:
%%writefile train/train_mult_worker_mirrored.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Run on multiple workers with GPU ###################
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

with strategy.scope():
###############################################################

    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with multiple GPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_mult_mirrored.h5', overwrite=True)

In [ ]:
%%writefile config/multi_worker_gpu.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 4 T4 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80
  # Configure 3 workers, each with 4 T4 GPUs
  workerCount: 3
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80

In [ ]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_worker_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_worker_mirrored \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/multi_worker_gpu.yaml